In [114]:
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Flatten,Dense,Dropout,Activation,BatchNormalization,Conv2D,MaxPooling2D
import os 
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [116]:
num_class = 2
img_row,img_col = 150,150
batch_size = 64

In [118]:
train_data_dir = '../datasets/gender_faces/train'
validation_data_dir = '../datasets/gender_faces/valid'

In [119]:
train_datagen = ImageDataGenerator( 
    rescale= 1./255,
    rotation_range=30,
    shear_range= 0.3,
    zoom_range= 0.4,
    width_shift_range= 0.4,
    height_shift_range= 0.4,
    horizontal_flip= True,
    fill_mode= 'nearest'
)

In [120]:
valid_datagen = ImageDataGenerator(rescale=1./255)

In [121]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode= 'grayscale',
    target_size= (img_row,img_col),
    batch_size= batch_size,
    class_mode= 'categorical',
    shuffle= True
)

Found 1600 images belonging to 2 classes.


In [122]:
validation_generator = train_datagen.flow_from_directory(
    validation_data_dir,
    color_mode= 'grayscale',
    target_size= (img_row,img_col),
    batch_size= batch_size,
    class_mode= 'categorical',
    shuffle= True
)

Found 340 images belonging to 2 classes.


In [123]:
model = Sequential()

In [124]:
model.add(
    Conv2D(32,(3,3),
    padding= 'same',kernel_initializer='he_normal',
    input_shape = (img_row,img_col,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(
    Conv2D(32,(3,3),
    padding= 'same',kernel_initializer='he_normal',
    input_shape = (img_row,img_col,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [125]:
model.add(
    Conv2D(64,(3,3),
    padding= 'same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(
    Conv2D(64,(3,3),
    padding= 'same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())  

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [126]:
model.add(
    Conv2D(128,(3,3),
    padding= 'same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(
    Conv2D(128,(3,3),
    padding= 'same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())  

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [127]:
model.add(
    Conv2D(256,(3,3),
    padding= 'same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(
    Conv2D(256,(3,3),
    padding= 'same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())  

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [128]:
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [129]:
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [130]:
model.add(Dense(num_class,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [131]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_65 (Conv2D)           (None, 150, 150, 32)      320       
_________________________________________________________________
activation_59 (Activation)   (None, 150, 150, 32)      0         
_________________________________________________________________
batch_normalization_53 (Batc (None, 150, 150, 32)      128       
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 150, 150, 32)      9248      
_________________________________________________________________
activation_60 (Activation)   (None, 150, 150, 32)      0         
_________________________________________________________________
batch_normalization_54 (Batc (None, 150, 150, 32)      128       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 75, 75, 32)       

In [132]:
checkpoint = ModelCheckpoint(
    'Gender_checker.h5',
    monitor='val_loss',
    mode = 'min',
    save_best_only=True,
    verbose = 1
)

In [133]:
earlystop = EarlyStopping( 
    monitor= 'val_loss',
    min_delta= 0,
    patience= 9,
    verbose = 1,
    restore_best_weights= True
)

In [134]:
reduce_lr = ReduceLROnPlateau( 
    monitor= 'val_loss',
    min_delta= 0.0001,
    patience= 3,
    verbose = 1,
    factor= 0.2
)

In [135]:
callbacks = [earlystop,checkpoint,reduce_lr]

In [136]:
model.compile(
            loss='categorical_crossentropy',
            optimizer= Adam(lr=0.001),
            metrics=['accuracy']
             
             )

In [137]:
epochs = 30
no_train_samples = 1600
no_validation_samples = 340

In [138]:
history =model.fit_generator(
        train_generator,
        steps_per_epoch= no_train_samples//batch_size,
        epochs = epochs,
        callbacks= callbacks,
        validation_data= validation_generator,
        validation_steps= no_validation_samples//batch_size
)

Epoch 1/30
25/25 [==============================] - 881s 35s/step - loss: 1.1356 - accuracy: 0.5537 - val_loss: 24.8169 - val_accuracy: 0.5031

Epoch 00001: val_loss improved from inf to 24.81688, saving model to Gender_checker.h5
Epoch 2/30
25/25 [==============================] - 752s 30s/step - loss: 0.9283 - accuracy: 0.5894 - val_loss: 8.0022 - val_accuracy: 0.4964

Epoch 00002: val_loss improved from 24.81688 to 8.00223, saving model to Gender_checker.h5
Epoch 3/30
25/25 [==============================] - 744s 30s/step - loss: 0.8712 - accuracy: 0.5962 - val_loss: 0.9612 - val_accuracy: 0.5290

Epoch 00003: val_loss improved from 8.00223 to 0.96121, saving model to Gender_checker.h5
Epoch 4/30
25/25 [==============================] - 750s 30s/step - loss: 0.7819 - accuracy: 0.6050 - val_loss: 2.2288 - val_accuracy: 0.5072

Epoch 00004: val_loss did not improve from 0.96121
Epoch 5/30
25/25 [==============================] - 749s 30s/step - loss: 0.7224 - accuracy: 0.6500 - val_lo